In [1]:
import nltk
import pandas as pd
from textblob import TextBlob
import logging
from gensim import corpora, models, similarities
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3
import lda
pd.set_option('display.max_colwidth', 500)

In [2]:
ltr=pd.read_csv('/Users/shani16/fletcher/data/ltr2.csv')

/Users/shani16/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (5,7,10,11,13,14,15,16,17,22,24,25,26,27,29,32,33,34,35,36,39,40,41,44,45,57,58,61,68,77,78,79,80,82,107,108,144,145,146,147,154,155,156,157,158,160,161,162,163,164,166,168,171,173,174,175,176,177,179) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
ltr.loc[ltr['Translation to English for: LTR Comment'].isnull(),'Translation to English for: LTR Comment'] = ltr['LTR Comment']

In [4]:
ltr=ltr[(ltr['Survey Type'] == "Insights")]
det= ltr[(ltr['LTR'] <= 6)]
df=det[['email', 'Translation to English for: LTR Comment']]
df.columns=['email', 'comment']
df=pd.DataFrame(df.dropna())

In [5]:
df.head()

,email,comment
0,mban@bpba.com.ar,I would be interested they can solucuonar inconvenience or problems faster.
6,simone.barbosa@lojasrenner.com.br,Meet as soon as the so-called demands. Improve the product as is requested via requests. Improving product performance.
8,mdunn@wellington.com,It has been a very unsatisfactory experience with CA with regard to the cancelation of our hosting agreement for Clarity.
9,andre_doles@gap.com,Not leave your customers hanging when they need you.
10,cecilia.shea@sabic-ip.com,"Contract Management, strategic planning & regular customer engagement."


In [6]:
df1=df

In [7]:
df1=df1.replace({'\r': ' '}, regex=True)
df1=df1.replace({'\xff': ' '}, regex=True)
df1=df1.replace({'\xfa': ' '}, regex=True)
df1=df1.replace({'\x84': ' '}, regex=True)
df1=df1.replace({'\x94': ' '}, regex=True)
df1=df1.replace({'\x87': ' '}, regex=True)
df1=df1.replace({'\xc60': ' '}, regex=True)
df1=df1.replace({'\xa2lio': ' '}, regex=True)
df1=df1.replace({'\xc6o': ' '}, regex=True)
# df2=df1.comment.tolist()
# tok=[nltk.sent_tokenize(s) for s in df2]

In [8]:
df1.head()

,email,comment
0,mban@bpba.com.ar,I would be interested they can solucuonar inconvenience or problems faster.
6,simone.barbosa@lojasrenner.com.br,Meet as soon as the so-called demands. Improve the product as is requested via requests. Improving product performance.
8,mdunn@wellington.com,It has been a very unsatisfactory experience with CA with regard to the cancelation of our hosting agreement for Clarity.
9,andre_doles@gap.com,Not leave your customers hanging when they need you.
10,cecilia.shea@sabic-ip.com,"Contract Management, strategic planning & regular customer engagement."


In [9]:
df1['token']=df1['comment'].apply(sent_tokenize)

In [10]:
df1.head()

,email,comment,token
0,mban@bpba.com.ar,I would be interested they can solucuonar inconvenience or problems faster.,[I would be interested they can solucuonar inconvenience or problems faster.]
6,simone.barbosa@lojasrenner.com.br,Meet as soon as the so-called demands. Improve the product as is requested via requests. Improving product performance.,"[Meet as soon as the so-called demands., Improve the product as is requested via requests., Improving product performance.]"
8,mdunn@wellington.com,It has been a very unsatisfactory experience with CA with regard to the cancelation of our hosting agreement for Clarity.,[It has been a very unsatisfactory experience with CA with regard to the cancelation of our hosting agreement for Clarity.]
9,andre_doles@gap.com,Not leave your customers hanging when they need you.,[Not leave your customers hanging when they need you.]
10,cecilia.shea@sabic-ip.com,"Contract Management, strategic planning & regular customer engagement.","[Contract Management, strategic planning & regular customer engagement.]"


In [11]:
df_new = df1.groupby('email').token.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)
df_new.columns = ['email','sentence']

In [12]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords += ['.', ',', '(', ')', "'", '"', 'ca']
stopwords += ['us', 'use', 'get', 'make', 'needs', 'need', 'customer','customers',
              'would', 'get', 'like', '30', 'also', 'many', 'day'
             '30', '1000', '10', '06', '100', '1000', '1000s', '11', '12', '13','14', '15', '18', '19', '1sr', '200',
             '2005', '2009', '2011', '2012', '20126', '2014', '2015', 'vs', 'you', 'we', 'know', 'even', 'see', 'zuk', 
              'fen' , 'gartner', 'rules', 'cas', 'year', 'nothing', 'look', 'sure', 'solutions', 'better', 'sure' ,
              'line', 'one','clear', 'companies', 'company', 'work', 'good', 'recommend', 'much', 'dont', 'new', 
              'technology', 'document', 'n/a']


In [13]:
# df_new['comment1']=pd.DataFrame(final_doc)

In [14]:
df_new.head()

,email,sentence
0,jmccullough3@csc.com,Better execution on production integration and ease of reporting
1,4254.jgodoy@bradesco.com.br,"the report says that this questionnaire can answer it until 22/06, but is allowing me to continue to fill it - lowering prices of their solutions - reduce solution times of your technical support team through the Support Center (formal processes) - availa"
2,Anandv78@gmail.com,We have been using the solution provided by CA Technologies in a critical function.
3,Anandv78@gmail.com,The one single area which requires immediate attention is to ensure overall improvement in support levels and timely response to organisational requirements.
4,Anandv78@gmail.com,Our experience has been very poor with prolonged resolutions for critical issues raised.


In [15]:
# stem1=df_new['sentence'].tolist()
# tok=[nltk.word_tokenize(s) for s in stem1]

from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer=CountVectorizer(stop_words=stopwords, ngram_range=(1,2))
counts=count_vectorizer.fit_transform(df_new['sentence'])

# 

In [16]:
num_clusters = 50  # sp.unique(labels).shape[0]

import nltk.stem
english_stemmer = nltk.stem.SnowballStemmer('english')

from sklearn.feature_extraction.text import TfidfVectorizer


class StemmedTfidfVectorizer(TfidfVectorizer):

    def build_analyzer(self):
        analyzer = super(TfidfVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))

vectorizer = StemmedTfidfVectorizer(min_df=10, max_df=0.5,
                                    stop_words=stopwords, decode_error='ignore'
                                    )
vectorized = vectorizer.fit_transform(df_new.sentence)



In [17]:
feature= vectorized.toarray()
feature_df = pd.DataFrame(feature, columns=vectorizer.get_feature_names())
feature_df[:1]

,abil,abl,access,account,across,activ,actual,ad,adapt,addit,...,websit,week,well,weve,within,without,work,world,year,yet
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
count_vectorizer1=TfidfVectorizer(stop_words=stopwords, ngram_range=(1,2))
counts1=count_vectorizer1.fit_transform(df_new['sentence'])

In [19]:
# feature= counts.toarray()
# feature_df = pd.DataFrame(feature, columns=count_vectorizer.get_feature_names())
# feature_df[:1]

In [20]:
word_frequency = feature_df.sum().reset_index()
word_frequency.columns=['word', 'freq']

word_frequency=word_frequency.sort(columns='freq', ascending= False)
word_frequency[:20]



/Users/shani16/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,word,freq
321,product,226.619673
424,support,148.910369
204,improv,122.239533
379,servic,71.394032
316,price,60.549757
215,integr,59.364131
327,provid,58.706619
222,issu,58.017831
438,time,56.516459
443,tool,53.281590


In [21]:
feature_df.to_csv("feature_df.csv", cols=['sum'])

In [22]:
feature1= counts1.toarray()
feature_df1 = pd.DataFrame(feature1, columns=count_vectorizer.get_feature_names())
feature_df1[:1]

,00137422,00137422 entm,10000,10000 mips,1st,1st support,2000,2000 physical,21st,21st century,...,youtube com,youve,youve changed,zeitl,zero,zero confidence,zone,zone consideration,zsecure,zsecure beta88
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
word_frequency1 = feature_df1.sum().reset_index()
word_frequency1.columns=['word', 'freq']

word_frequency1=word_frequency1.sort(columns='freq', ascending= False)
word_frequency1[:50]

/Users/shani16/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,word,freq
13607,products,71.644222
17992,support,68.724507
13346,product,55.688841
8191,improve,46.662519
14373,quality,23.196352
16972,software,22.888354
9225,issues,22.675108
14084,provide,21.492348
19038,time,20.506324
16368,service,20.051511


## Clusters K Means

In [24]:
from sklearn.cluster import KMeans
model = KMeans (n_clusters=5, init= 'k-means++')
model.fit(vectorized) #add sparse matrix 

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=5, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [25]:
#sorting in descending order ie: -1
#giving top ten words ie: :10
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(3):
   print "Cluster %d:" % i,
   for ind in order_centroids[i, :10]:
       print ' %s' % terms[ind],
   print

Top terms per cluster:
Cluster 0:  tool  time  issu  servic  improv  manag  softwar  cost  team  work
Cluster 1:  provid  product  support  servic  inform  cost  solut  time  manag  document
Cluster 2:  support  improv  product  technic  servic  qualiti  team  knowledg  issu  respons


In [26]:
clusters=pd.Series(model.labels_)
df_new['clusters']=clusters

In [27]:
model1 = KMeans (n_clusters=5, init= 'k-means++')
model1.fit(counts1) #add sparse matrix 

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=5, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [28]:
clusters1=pd.Series(model1.labels_)

In [29]:
#sorting in descending order ie: -1
#giving top ten words ie: :10
print("Top terms per cluster:")
order_centroids = model1.cluster_centers_.argsort()[:, ::-1]
terms = count_vectorizer1.get_feature_names()
for i in range(3):
   print "Cluster %d:" % i,
   for ind in order_centroids[i, :10]:
       print ' %s' % terms[ind],
   print

Top terms per cluster:
Cluster 0:  time  issues  tools  improve  pricing  business  service  account  provide  team
Cluster 1:  support  improve  improve support  technical support  technical  support products  products  quality  service  issues
Cluster 2:  software  lower  prices  costs  lower prices  reduce  cost  lower costs  price  pricing


In [30]:
df_new['clusters1']=clusters1

In [31]:
df_new.head()

,email,sentence,clusters,clusters1
0,jmccullough3@csc.com,Better execution on production integration and ease of reporting,3,0
1,4254.jgodoy@bradesco.com.br,"the report says that this questionnaire can answer it until 22/06, but is allowing me to continue to fill it - lowering prices of their solutions - reduce solution times of your technical support team through the Support Center (formal processes) - availa",2,1
2,Anandv78@gmail.com,We have been using the solution provided by CA Technologies in a critical function.,1,0
3,Anandv78@gmail.com,The one single area which requires immediate attention is to ensure overall improvement in support levels and timely response to organisational requirements.,0,0
4,Anandv78@gmail.com,Our experience has been very poor with prolonged resolutions for critical issues raised.,0,0


In [32]:
df_new.groupby('clusters1').sum()

,clusters
clusters1,
0,467
1,454
2,185
3,1118
4,855


In [33]:
df_new.groupby('clusters').sum()

,clusters1
clusters,
0,397
1,167
2,366
3,2073
4,120


### Agglomerative clustering 

## DBSCAN

In [34]:
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler


In [35]:
db = DBSCAN(eps=0.9, min_samples=8).fit(vectorized)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

print('Estimated number of clusters: %d' % n_clusters_)

Estimated number of clusters: 21


In [36]:
# clust = DBSCAN(eps=0.1, min_samples=5, metric="precomputed")
# clust.fit(counts)

# # print cluster report
# stitles = []
# ftitles = open(os.path.join(OUTPUT_DIR, "stitles.txt"), 'rb')
# for line in ftitles:
#     stitles.append(line.strip().split("\t")[0])
# ftitles.close()

# preds = clust.labels_
# clabels = np.unique(preds)
# for i in range(clabels.shape[0]):
#     if clabels[i] < 0:
#         continue
#     cmem_ids = np.where(preds == clabels[i])[0]
#     cmembers = []
#     for cmem_id in cmem_ids:
#         cmembers.append(stitles[cmem_id])
#     print "Cluster#%d: %s" % (i, ", ".join(cmembers))


In [37]:
db

DBSCAN(algorithm='auto', eps=0.9, leaf_size=30, metric='euclidean',
    min_samples=8, p=None, random_state=None)

## LDA

In [38]:
from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

n_topics = 7
n_top_words = 12


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

stopwords = nltk.corpus.stopwords.words('english')
stopwords += ['.', ',', '(', ')', "'", '"', 'ca']
stopwords += ['us', 'use', 'get', 'make', 'needs', 'need', 'customer','customers',
              'would', 'get', 'like', '30', 'also', 'many', 'day'
             '30', '1000', '10', '06', '100', '1000', '1000s', '11', '12', '13','14', '15', '18', '19', '1sr', '200',
             '2005', '2009', '2011', '2012', '20126', '2014', '2015', 'vs', 'you', 'we', 'know', 'even', 'see', 'zuk', 
              'fen' , 'gartner', 'rules', 'cas', 'year', 'nothing', 'look', 'sure', 'solutions', 'better', 'sure' ,
              'line', 'one','clear', 'products', 'product']
print( "Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, #max_features=n_features,
                                   stop_words=stopwords)
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(df_new['sentence'])
print("done in %0.3fs." % (time() - t0))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
lda = LatentDirichletAllocation(n_topics=n_topics,
                                random_state=222)
t0 = time()
lda.fit(vectorized)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Extracting tf-idf features for NMF...
done in 0.086s.
Extracting tf features for LDA...
done in 4.665s.

Topics in LDA model:
Topic #0:
respons implement work time partner point request area meet product contact profession
Topic #1:
improv product support integr qualiti servic function tool user technic deliv communic
Topic #2:
difficult year engag experi document easier enhanc innov support instal product set
Topic #3:
price product provid help valu solut lower technolog understand base softwar term
Topic #4:
cost product team account busi offer manag issu train process purchas reduc
Topic #5:
licens flexibl releas product direct model way contract roadmap feel deploy strategi
Topic #6:
current requir case focus problem upgrad still want involv product go version



In [39]:
test=pd.DataFrame(lda.transform(vectorized))

In [40]:
test.head()

,0,1,2,3,4,5,6
0,0.142886,1.727615,0.700000,0.142895,0.143552,0.143439,0.142879
1,0.143131,0.143377,0.143037,0.800068,3.188810,0.142937,0.143284
2,0.142903,0.143724,0.143138,2.557317,0.143421,0.143044,0.142937
3,0.745644,0.877082,0.505125,0.143032,0.915892,0.143081,0.995109
4,0.616349,0.143119,2.084846,0.144372,0.143003,0.142858,0.142857


In [41]:
topics = ['Helps be succesful', 'Support', 'Installation and Enhancements', 'Pricing', 'Acount Team Mgt',
              'Flexibility: release and licensing', 
              'Upgrades and Implementation']

In [42]:
# test.columns=['Helps be succesful', 'Support', 'Installation and Enhancements', 'Pricing', 'Acount Team Mgt',
#               'Flexibility: release and licensing', 
#               'Upgrades and Implementation']

In [43]:
test.head()

,0,1,2,3,4,5,6
0,0.142886,1.727615,0.700000,0.142895,0.143552,0.143439,0.142879
1,0.143131,0.143377,0.143037,0.800068,3.188810,0.142937,0.143284
2,0.142903,0.143724,0.143138,2.557317,0.143421,0.143044,0.142937
3,0.745644,0.877082,0.505125,0.143032,0.915892,0.143081,0.995109
4,0.616349,0.143119,2.084846,0.144372,0.143003,0.142858,0.142857


In [44]:
test.describe()

,0,1,2,3,4,5,6
count,3295.000000,3295.000000,3295.000000,3295.000000,3295.000000,3295.000000,3295.000000
mean,0.341897,0.538495,0.390498,0.441985,0.477499,0.329335,0.423106
std,0.416088,0.596987,0.463654,0.526297,0.552345,0.399287,0.502914
min,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857
25%,0.142857,0.142857,0.142874,0.142917,0.142922,0.142857,0.142889
50%,0.143011,0.143549,0.143040,0.143200,0.143239,0.142940,0.143069
75%,0.145294,0.887383,0.561422,0.662510,0.740547,0.144267,0.635503
max,3.334767,2.939337,2.885832,3.240330,3.515719,3.103633,3.497695


In [45]:
def create_topics(cutoffs, topic_names, df):
    for idx, topic in enumerate(topic_names):
        df[topic] = df[idx].apply(lambda x: 1 if x >= cutoffs[idx] else 0)
    return df

In [46]:
percent= [0.341897, 0.538495, 0.390498, 0.441985, 0.477499, 0.329335, 0.423106]

In [47]:
topics

['Helps be succesful',
 'Support',
 'Installation and Enhancements',
 'Pricing',
 'Acount Team Mgt',
 'Flexibility: release and licensing',
 'Upgrades and Implementation']

In [48]:
test1=create_topics(percent, topics, test)

In [49]:
test1.head()

,0,1,2,3,4,5,6,Helps be succesful,Support,Installation and Enhancements,Pricing,Acount Team Mgt,Flexibility: release and licensing,Upgrades and Implementation
0,0.142886,1.727615,0.700000,0.142895,0.143552,0.143439,0.142879,0,1,1,0,0,0,0
1,0.143131,0.143377,0.143037,0.800068,3.188810,0.142937,0.143284,0,0,0,1,1,0,0
2,0.142903,0.143724,0.143138,2.557317,0.143421,0.143044,0.142937,0,0,0,1,0,0,0
3,0.745644,0.877082,0.505125,0.143032,0.915892,0.143081,0.995109,1,1,1,0,1,0,1
4,0.616349,0.143119,2.084846,0.144372,0.143003,0.142858,0.142857,1,0,1,0,0,0,0


In [50]:
test1['n_topics'] = test.ix[: , 8:].sum(axis=1) 

In [51]:
test1.head()

,0,1,2,3,4,5,6,Helps be succesful,Support,Installation and Enhancements,Pricing,Acount Team Mgt,Flexibility: release and licensing,Upgrades and Implementation,n_topics
0,0.142886,1.727615,0.700000,0.142895,0.143552,0.143439,0.142879,0,1,1,0,0,0,0,2
1,0.143131,0.143377,0.143037,0.800068,3.188810,0.142937,0.143284,0,0,0,1,1,0,0,2
2,0.142903,0.143724,0.143138,2.557317,0.143421,0.143044,0.142937,0,0,0,1,0,0,0,1
3,0.745644,0.877082,0.505125,0.143032,0.915892,0.143081,0.995109,1,1,1,0,1,0,1,4
4,0.616349,0.143119,2.084846,0.144372,0.143003,0.142858,0.142857,1,0,1,0,0,0,0,1


In [52]:
test1['n_topics'].value_counts()

2    1172
1    1152
3     578
0     235
4     141
5      16
6       1
Name: n_topics, dtype: int64

8% of comments don't have a topic

In [53]:
test1=test1[['Helps be succesful', 'Support', 'Installation and Enhancements', 'Pricing', 'Acount Team Mgt',
              'Flexibility: release and licensing', 
              'Upgrades and Implementation', 'n_topics']]

In [54]:
df_new1=df_new.join(test1)

In [55]:
df_new1.head()

,email,sentence,clusters,clusters1,Helps be succesful,Support,Installation and Enhancements,Pricing,Acount Team Mgt,Flexibility: release and licensing,Upgrades and Implementation,n_topics
0,jmccullough3@csc.com,Better execution on production integration and ease of reporting,3,0,0,1,1,0,0,0,0,2
1,4254.jgodoy@bradesco.com.br,"the report says that this questionnaire can answer it until 22/06, but is allowing me to continue to fill it - lowering prices of their solutions - reduce solution times of your technical support team through the Support Center (formal processes) - availa",2,1,0,0,0,1,1,0,0,2
2,Anandv78@gmail.com,We have been using the solution provided by CA Technologies in a critical function.,1,0,0,0,0,1,0,0,0,1
3,Anandv78@gmail.com,The one single area which requires immediate attention is to ensure overall improvement in support levels and timely response to organisational requirements.,0,0,1,1,1,0,1,0,1,4
4,Anandv78@gmail.com,Our experience has been very poor with prolonged resolutions for critical issues raised.,0,0,1,0,1,0,0,0,0,1


In [56]:
df_new1['n_topics'].value_counts()

2    1172
1    1152
3     578
0     235
4     141
5      16
6       1
Name: n_topics, dtype: int64

In [65]:
df_new1['Helps be succesful'].value_counts()

0    2509
1     786
Name: Helps be succesful, dtype: int64

In [66]:
df_new1['Installation and Enhancements'].value_counts()

0    2372
1     923
Name: Installation and Enhancements, dtype: int64

In [57]:
df_new1['Acount Team Mgt'].value_counts()

0    2215
1    1080
Name: Acount Team Mgt, dtype: int64

In [58]:
df_new1['Pricing'].value_counts()

0    2307
1     988
Name: Pricing, dtype: int64

In [59]:
df_new1['Flexibility: release and licensing'].value_counts()

0    2544
1     751
Name: Flexibility: release and licensing, dtype: int64

In [60]:
df_new1['Support'].value_counts()

0    2127
1    1168
Name: Support, dtype: int64

In [61]:
df_new1['Upgrades and Implementation'].value_counts()

0    2325
1     970
Name: Upgrades and Implementation, dtype: int64

In [62]:
df_new1['Flexibility: release and licensing'].value_counts()

0    2544
1     751
Name: Flexibility: release and licensing, dtype: int64

In [63]:
df_new1['Upgrades and Implementation'].value_counts()

0    2325
1     970
Name: Upgrades and Implementation, dtype: int64

In [ ]:
from vaderSentiment.vaderSentiment import sentiment as vaderSentiment

In [ ]:
def compound(row):
    return(vaderSentiment(row)['compound'])

In [ ]:
df_new1['compound']= df_new1['sentence'].apply(compound)

In [ ]:
df_new1['compound'].describe()

In [ ]:
df_new1

In [ ]:
df_new1.sort('compound')

In [ ]:
df_new1.to_csv("test.csv")

In [ ]:
test12=df_new1['sentence'].tolist()

In [ ]:
vaderSentiment(df_new1['sentence'][10])

In [ ]:
for a in test12:
    print vaderSentiment(a)['compound']

In [ ]:
df_new1.head()

In [ ]:
df_new1['sentence'][0]

In [ ]:
from os import path
from wordcloud import WordCloud


# Read the whole text.
text = open(('feature_df.csv')).read()

# Generate a word cloud image
wordcloud = WordCloud().generate(text)

# Display the generated image:
# the matplotlib way:
import matplotlib.pyplot as plt
plt.imshow(wordcloud)
plt.axis("off")

# take relative word frequencies into account, lower max_font_size
wordcloud = WordCloud(max_font_size=40, relative_scaling=.5).generate(text)
plt.figure()
plt.imshow(wordcloud)
plt.axis("off")
plt.show()
